In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.io as scio
from matplotlib.font_manager import FontProperties

font_path = r'C:\Windows\Fonts\simsun.ttc'
chinese_font = FontProperties(fname=r'C:\Windows\Fonts\simsun.ttc')
english_font = FontProperties(fname=r'C:\Windows\Fonts\times.ttf')
pd.options.display.float_format = '{:.3f}'.format

# 定义索引
rural = np.arange(0, 60, 2)
urban = np.arange(1, 60, 2)
east = np.r_[0, 1, 2, 3, 4, 5, 16, 17, 18, 19, 20, 21, 24, 25, 28, 29, 36, 37, 40, 41]
middle = np.r_[6, 7, 22, 23, 26, 27, 30, 31, 32, 33, 34, 35]
west = np.r_[8, 9, 10, 11, 12, 13, 14, 15, 38, 39, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]

# 定义计算基尼系数的函数
def gini_index(array):
    cum_wealths = np.cumsum(sorted(np.append(array, 0)))
    sum_wealths = cum_wealths[-1]
    xarray = np.array(range(0, len(cum_wealths))) / float(len(cum_wealths)-1)
    yarray = cum_wealths / sum_wealths
    B = np.trapz(yarray, x=xarray)
    A = 0.5 - B
    return A / (A + B)

# 定义计算泰尔指数的函数
def theil_index(array):
    mean_income = np.mean(array)
    income_ratio = array / mean_income
    log_income_ratio = np.log(income_ratio, where=(income_ratio!=0))
    contribution = income_ratio * log_income_ratio
    return np.mean(contribution)

In [3]:
# 读取基准情形的变量和参数
equilibrium2010 = np.load('./data/equilibrium2010.npz')
R = equilibrium2010['R']
X = equilibrium2010['X']
m = equilibrium2010['m']
pi = equilibrium2010['pi']
L = equilibrium2010['L']
L_bar = equilibrium2010['L_bar']
K = equilibrium2010['K']
w = equilibrium2010['w']
P = equilibrium2010['P']
W = equilibrium2010['W']

parameter = np.load('./data/Parameter.npz')
kappa = parameter['kappa']
theta = parameter['theta']
beta = parameter['beta']
psi_ag = parameter['psi_ag']
psi_na = parameter['psi_na']
alpha_ag = parameter['alpha_ag']
alpha_na = parameter['alpha_na']
sigma_ag = parameter['sigma_ag']
sigma_na = parameter['sigma_na']
eta_ag_ag = parameter['eta_ag_ag']
eta_ag_na = parameter['eta_ag_na']
eta_na_ag = parameter['eta_na_ag']
eta_na_na = parameter['eta_na_na']
phiu = parameter['phiu']
phip = parameter['phip']
chi = parameter['chi']
eta = parameter['eta']
delta = parameter['delta']
xi_ag = parameter['xi_ag']
xi_na = parameter['xi_na']
t = parameter['t']

In [4]:
# 导入内生变量的反事实变动
counterfactuals = np.load('./data/Counterfactuals2010.npz')

dw = counterfactuals['dw']
dL = counterfactuals['dL']
dp = counterfactuals['dp']
dG = counterfactuals['dG']
dc = counterfactuals['dc']
dpi = counterfactuals['dpi']
pi_prime = counterfactuals['pi_prime']
dP = counterfactuals['dP']
dW = counterfactuals['dW']
dmu = counterfactuals['dmu']
dm = counterfactuals['dm']
m_prime = counterfactuals['m_prime']
R_prime = counterfactuals['R_prime']
X_prime = counterfactuals['X_prime']
ddelta = counterfactuals['ddelta']
drh = counterfactuals['drh']

coef_rhKh_ag = (1 - beta) * (1 - t) * (alpha_ag + sigma_ag) / beta / alpha_ag
coef_rhKh_na = (1 - beta) * (1 - t)

w_prime = w * dw
L_prime = L * dL
W_prime = W * dW
P_prime = P * dP
dg = dG / (dL ** chi)
rhKh = ((w * L).reshape([30, 2]) * np.array([coef_rhKh_ag, coef_rhKh_na]).reshape([1, 2])).reshape([60, 1])
rhKh_prime = rhKh * dw * dL

In [8]:
# 生成汇总表格

## 总产出
dX_country = ((X_prime / P_prime).sum() / (X / P).sum() - 1) * 100
dX_urban = ((X_prime / P_prime)[urban].sum() / (X / P)[urban].sum() - 1) * 100
dX_rural = ((X_prime / P_prime)[rural].sum() / (X / P)[rural].sum() - 1) * 100
dX_east = ((X_prime / P_prime)[east].sum() / (X / P)[east].sum() - 1) * 100
dX_middle = ((X_prime / P_prime)[middle].sum() / (X / P)[middle].sum() - 1) * 100
dX_west = ((X_prime / P_prime)[west].sum() / (X / P)[west].sum() - 1) * 100

## 人口数
dL_urban = (L_prime[urban].sum() / L[urban].sum() - 1) * 100
dL_rural = (L_prime[rural].sum() / L[rural].sum() - 1) * 100
dL_east = (L_prime[east].sum() / L[east].sum() - 1) * 100
dL_middle = (L_prime[middle].sum() / L[middle].sum() - 1) * 100
dL_west = (L_prime[west].sum() / L[west].sum() - 1) * 100

## 人均实际GDP
dW_country = (((W_prime * L_prime).sum() / L_prime.sum()) / ((W * L).sum() / L.sum()) - 1) * 100
dW_urban = (((W_prime * L_prime)[urban].sum() / L_prime[urban].sum()) / ((W * L)[urban].sum() / L[urban].sum()) - 1) * 100
dW_rural = (((W_prime * L_prime)[rural].sum() / L_prime[rural].sum()) / ((W * L)[rural].sum() / L[rural].sum()) - 1) * 100
dW_east = (((W_prime * L_prime)[east].sum() / L_prime[east].sum()) / ((W * L)[east].sum() / L[east].sum()) - 1) * 100
dW_middle = (((W_prime * L_prime)[middle].sum() / L_prime[middle].sum()) / ((W * L)[middle].sum() / L[middle].sum()) - 1) * 100
dW_west = (((W_prime * L_prime)[west].sum() / L_prime[west].sum()) / ((W * L)[west].sum() / L[west].sum()) - 1) * 100

## 人均公共支出
coef_g_na = 1 - beta + sigma_na / alpha_na + beta * t
coef_g_ag = (alpha_ag + sigma_ag) * t / (beta + t - beta * t) / alpha_ag
dg_country = (
        (((w_prime * L_prime / P_prime)[urban] * coef_g_na).sum() + ((w_prime * L_prime / P_prime)[rural] * coef_g_ag).sum()) / 
        (((w * L / P)[urban] * coef_g_na).sum() + ((w * L / P)[rural] * coef_g_ag).sum())
    - 1) * 100
dg_urban = (
        ((w_prime * L_prime / P_prime)[urban].sum() / (L_prime[urban].sum() ** chi)) / 
        ((w * L / P)[urban].sum() / (L[urban].sum() ** chi)) 
    - 1) * 100
dg_rural = (
        ((w_prime * L_prime / P_prime)[rural].sum() / (L_prime[rural].sum() ** chi)) / 
        ((w * L / P)[rural].sum() / (L[rural].sum() ** chi)) 
    - 1) * 100

## 房价
drh_country = ((drh / dP).mean() - 1) * 100
drh_urban = ((drh / dP)[urban].mean() - 1) * 100
drh_rural = ((drh / dP)[rural].mean() - 1) * 100
drh_east = ((drh / dP)[east].mean() - 1) * 100
drh_middle = ((drh / dP)[middle].mean() - 1) * 100
drh_west = ((drh / dP)[west].mean() - 1) * 100

## 人均实际GDP基尼系数
dincgini_country = (gini_index(W_prime) / gini_index(W) - 1) * 100
dincgini_urban = (gini_index(W_prime[urban]) / gini_index(W[urban]) - 1) * 100
dincgini_rural = (gini_index(W_prime[rural]) / gini_index(W[rural]) - 1) * 100
dincgini_east = (gini_index(W_prime[east]) / gini_index(W[east]) - 1) * 100
dincgini_middle = (gini_index(W_prime[middle]) / gini_index(W[middle]) - 1) * 100
dincgini_west = (gini_index(W_prime[west]) / gini_index(W[west]) - 1) * 100

## 本地贸易份额
dpiii_country = ((np.diag(pi_prime) - np.diag(pi)) * 100).mean()
dpiii_urban = ((np.diag(pi_prime)[urban] - np.diag(pi)[urban]) * 100).mean()
dpiii_rural = ((np.diag(pi_prime)[rural] - np.diag(pi)[rural]) * 100).mean()
dpiii_east = ((np.diag(pi_prime)[east] - np.diag(pi)[east]) * 100).mean()
dpiii_middle = ((np.diag(pi_prime)[middle] - np.diag(pi)[middle]) * 100).mean()
dpiii_west = ((np.diag(pi_prime)[west] - np.diag(pi)[west]) * 100).mean()

table = pd.DataFrame()
table = table.append(pd.Series([dX_country, dX_urban, dX_rural, dX_east, dX_middle, dX_west]), ignore_index=True)
table = table.append(pd.Series([np.nan, dL_urban, dL_rural, dL_east, dL_middle, dL_west]), ignore_index=True)
table = table.append(pd.Series([dW_country, dW_urban, dW_rural, dW_east, dW_middle, dW_west]), ignore_index=True)
table = table.append(pd.Series([dg_country, dg_urban, dg_rural]), ignore_index=True)
table = table.append(pd.Series([drh_country, drh_urban, drh_rural, drh_east, drh_middle, drh_west]), ignore_index=True)
table = table.append(pd.Series([dincgini_country, dincgini_urban, dincgini_rural, np.nan, np.nan, np.nan]), ignore_index=True)
table = table.append(pd.Series([dpiii_country, dpiii_urban, dpiii_rural, dpiii_east, dpiii_middle, dpiii_west]), ignore_index=True)

table.columns = ['全国', '城市', '农村', '东部', '中部', '西部']
table.index = ['总产出的百分比变化', '常住居民数量的百分比变化', '人均实际GDP的百分比变化', '人均公共支出的百分比变化', '房价的百分比变化', '省际人均实际GDP基尼系数的百分比变化', '本地贸易份额的百分比变化']
table

,全国,城市,农村,东部,中部,西部
总产出的百分比变化,2.184,2.184,2.177,3.254,0.336,1.595
常住居民数量的百分比变化,NaN,6.975,-10.232,4.937,-5.197,-1.909
人均实际GDP的百分比变化,4.226,-2.261,14.322,1.555,6.207,3.812
人均公共支出的百分比变化,4.311,-1.600,13.095,NaN,NaN,NaN
房价的百分比变化,2.761,2.606,2.915,3.903,0.996,2.701
省际人均实际GDP基尼系数的百分比变化,-9.333,-3.349,13.008,NaN,NaN,NaN
本地贸易份额的百分比变化,-0.131,-0.004,-0.257,-0.332,-0.138,0.017


In [6]:
# 根据人口流入规划划分前30%和后70%

dL_df = pd.DataFrame(dL[urban], columns=['dL'])
dL_df = dL_df.sort_values(by='dL', ascending=False)
high_flow_in = np.array(dL_df.index[:9].sort_values())
low_flow_in = np.array(dL_df.index[9:].sort_values())
high_flow_in, low_flow_in

(array([ 0,  1,  2,  3,  8, 10, 12, 18, 20], dtype=int64),
 array([ 4,  5,  6,  7,  9, 11, 13, 14, 15, 16, 17, 19, 21, 22, 23, 24, 25,
        26, 27, 28, 29], dtype=int64))

In [7]:
(gini_index(W_prime) / gini_index(W) - 1) * 100, (gini_index(W_prime[urban]) / gini_index(W[urban]) - 1) * 100, (gini_index(W_prime[rural]) / gini_index(W[rural]) - 1) * 100, (gini_index([W_prime[urban].mean(), W_prime[rural].mean()]) / gini_index([W[urban].mean(), W[rural].mean()]) - 1) * 100

(-9.332961919521576,
 -3.3494030074136316,
 13.007805889980428,
 -16.35822096465911)